In [19]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import yfinance as yf

# Pobieranie danych z Yahoo Finance
symbol = 'EURUSD=X'
start_date = '2010-01-01'
end_date = '2022-02-28'
data = yf.download(symbol, start=start_date, end=end_date)
# Przetwarzanie danych
data['Change'] = data['Close'].diff() # Kolumna 'Change' zawiera zmiany w cenie (zamknięcie - poprzednie zamknięcie)
data.dropna(inplace=True)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Change
Date,,,,,,,
2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,0,0.003404
2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,0,-0.005802
2010-01-06,1.436596,1.443460,1.429123,1.440403,1.440403,0,0.003807
2010-01-07,1.440300,1.444481,1.430206,1.431803,1.431803,0,-0.008600
2010-01-08,1.432090,1.443606,1.427104,1.441109,1.441109,0,0.009306
...,...,...,...,...,...,...,...
2022-02-21,1.131734,1.138563,1.131414,1.131721,1.131721,0,-0.004927
2022-02-22,1.130966,1.136377,1.128910,1.130966,1.130966,0,-0.000755
2022-02-23,1.133055,1.135976,1.130736,1.132913,1.132913,0,0.001948


In [10]:

X = np.array(data['Close']).reshape(-1, 1)
y = np.array(data['Change']).reshape(-1, 1)

# Dodanie dodatkowych obserwacji
X = np.repeat(X, 3, axis=1)

X

array([[1.44239783, 1.44239783, 1.44239783],
       [1.43659592, 1.43659592, 1.43659592],
       [1.44040334, 1.44040334, 1.44040334],
       ...,
       [1.13291335, 1.13291335, 1.13291335],
       [1.13056946, 1.13056946, 1.13056946],
       [1.11906898, 1.11906898, 1.11906898]])

In [11]:
# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train.shape

(2532, 3)

In [26]:
import tensorflow as tf

# Definiowanie modelu
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(None, 2532, 3)))
model.add(tf.keras.layers.Flatten())
model.add(MaxPooling1D(pool_size=1)) # zmiana pool_size na 1
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, verbose=1)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Błąd średniokwadratowy (MSE): {mse}')


ValueError: Input 0 of layer "max_pooling1d_8" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, None)